# Prerequisites 

In [1]:

import numpy as np


import numpy as np
from math import pi
import matplotlib.pyplot as plt


import json
import os


In [2]:
measurement_points=np.arange(0,102,4)
# time=measurement_points*102*dt*1e6
# time

state_list=['00','01','0+','+0','++']

qubit_list=[
    [21,22],
]

UR_list=['None','Both','0','1']

experiment_date = '06-01-25-CZ'


In [3]:
experiment_date = '06-01-25-CZ'


In [4]:
import os

# List all entries (files and directories) in current working directory
entries = os.listdir()
for name in entries:
    print(name)


circuit_list_Both_06-01-25-CZ.json
circuit_list_1_06-01-25-CZ.json
fitting-saurav.ipynb
.DS_Store
circuit_list_0_06-01-25-CZ.json
06-01-25-CZ
circuit_list_None_06-01-25-CZ.json


# Analysis

In [5]:
circuit_list={}

for ur in UR_list:
    ur_key=f'{ur}'
# …later, load it back
    with open(f'circuit_list_{ur_key}_{experiment_date}.json', 'r') as f:
        circuit_list[ur_key] = json.load(f)

In [6]:
import json
import os

# Example inputs; these should already be defined in your context:
# UR_list = [...]
# circuit_list = { ur_key: [exp1, exp2, ...], ... }
# experiment_date = "2025-06-20"  # for example, or however you define it
base_dir = os.getcwd()

res_dic = {}

for ur_key in UR_list:
    # Construct the directory where results were saved for this UR
    dir_path = os.path.join(base_dir, experiment_date, ur_key)
    if not os.path.isdir(dir_path):
        # Handle missing directory as appropriate: you may skip or raise an error.
        raise FileNotFoundError(f"Expected directory not found: {dir_path}")
    
    # For possible per-UR grouping if desired
    res_dic_save = {}
    
    for exp in circuit_list.get(ur_key, []):
        filename = f"results-{exp}.json"
        file_path = os.path.join(dir_path, filename)
        
        if not os.path.isfile(file_path):
            # Handle missing file: skip, warn, or raise
            raise FileNotFoundError(f"Expected file not found: {file_path}")
        
        with open(file_path, "r") as f:
            counts = json.load(f)
        
        # Store in the per-UR dictionary
        res_dic_save[exp] = counts
        
        # Store in the global flat dictionary, matching your saving key
        res_dic[f"{ur_key}-{exp}"] = counts
    
    # If you wish to keep a mapping per UR, you can store res_dic_save:
    # per_UR_results[ur_key] = res_dic_save
    # (Define per_UR_results = {} before the loop if needed.)


In [7]:
import math

def fid_CT(counts_in):
    """
    From a dict of 2-bit measurement counts, compute:
      - counts_00: raw shots where the outcome is '00'
      - p_00:      probability of '00'
      - sigma_00:  binomial std. deviation of p_00
    Returns (p_00, sigma_00, counts_00).
    """
    total_counts = sum(counts_in.values())
    if total_counts == 0:
        raise ValueError("No counts provided (sum is zero).")

    # Raw count for exactly '00'
    counts_00 = counts_in.get('00', 0)

    # Probability and its binomial σ
    p_00 = counts_00 / total_counts
    sigma_00 = math.sqrt(p_00 * (1 - p_00) / total_counts)

    return p_00, sigma_00


In [8]:
# assume fid_CT and fid_GHZ are already defined, and res_dic is your input dict

fid = {}
err = {}

for key, counts in res_dic.items():
        p, sigma = fid_CT(counts)
        fid[key] = p
        err[key] = sigma

# now fid and err hold your probabilities and uncertainties, e.g.
print(fid)
print(err)


{'None-00-0': 0.997, 'None-01-0': 0.9975, 'None-0+-0': 0.997, 'None-+0-0': 0.993, 'None-++-0': 0.994, 'None-00-4': 0.998, 'None-01-4': 0.995, 'None-0+-4': 0.978, 'None-+0-4': 0.946, 'None-++-4': 0.9395, 'None-00-8': 0.9975, 'None-01-8': 0.9855, 'None-0+-8': 0.959, 'None-+0-8': 0.8865, 'None-++-8': 0.886, 'None-00-12': 0.9985, 'None-01-12': 0.986, 'None-0+-12': 0.9445, 'None-+0-12': 0.79, 'None-++-12': 0.791, 'None-00-16': 0.999, 'None-01-16': 0.985, 'None-0+-16': 0.949, 'None-+0-16': 0.6765, 'None-++-16': 0.7275, 'None-00-20': 0.9965, 'None-01-20': 0.982, 'None-0+-20': 0.9305, 'None-+0-20': 0.5685, 'None-++-20': 0.6405, 'None-00-24': 0.9965, 'None-01-24': 0.976, 'None-0+-24': 0.902, 'None-+0-24': 0.4395, 'None-++-24': 0.5505, 'None-00-28': 0.9985, 'None-01-28': 0.9715, 'None-0+-28': 0.8935, 'None-+0-28': 0.3385, 'None-++-28': 0.4565, 'None-00-32': 0.999, 'None-01-32': 0.9685, 'None-0+-32': 0.888, 'None-+0-32': 0.213, 'None-++-32': 0.36, 'None-00-36': 0.9965, 'None-01-36': 0.9655, 'None

# Fitting

In [9]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# # Change to your desired output directory
# path = f"/Users/arian/Desktop/{experiment_date}-res/"
path=base_dir = os.getcwd()
os.makedirs(path, exist_ok=True)

# # Now change into it
# os.chdir(path)

# Matplotlib global settings
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'Apple LiSung'
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}\usepackage{braket}'

# Font settings for titles and axis labels
font_properties = {
    'fontsize': 18,
    'fontname': 'Times New Roman'
}

# Example model function: a + b * exp(-t / T_D) * cos(omega * t)
def damped_cosine(t, a, b, T_D, omega):
    return a + b * np.exp(-t / T_D) * np.cos(omega * t)

# Your data (assumed to be defined already)
# fid = { ... }
# err = { ... }

measurement_points = np.arange(0, 102, 4)
# state_list = ['00','01','0+','+0','++']
# UR_list = [...]
# err and fid are dicts mapping keys like f'{ur}-{state}-{mp}'

# Nine distinct markers for the nine states
markers = ['o', 's', '^', 'v', 'D', 'p', '*', 'x', '+']

for ur in UR_list:
    # create a new figure + axes
    fig, ax = plt.subplots(figsize=(8, 5))
    
    for st_ix, state in enumerate(state_list):
        # Prepare data arrays
        t_data = np.array(measurement_points, dtype=float)
        y_data = np.array([fid[f'{ur}-{state}-{mp}'] for mp in measurement_points], dtype=float)
        y_err  = np.array([err[f'{ur}-{state}-{mp}'] for mp in measurement_points], dtype=float)
        
        # Plot the raw data points (errorbars) without connecting lines
        eb = ax.errorbar(
            t_data,
            y_data,
            yerr=y_err,
            marker=markers[st_ix],
            linestyle='None',        # no line between points
            label=f'$|{state}\\rangle$',
            capsize=3,
        )
        # Extract the color that matplotlib chose for this dataset:
        # Errorbar returns an ErrorbarContainer; its `.lines` attribute 
        # (in recent matplotlib versions) gives a list whose first element is the Line2D for the data points.
        try:
            color = eb.lines[0].get_color()
        except Exception:
            # Fallback: let matplotlib cycle the same color for the fit automatically
            color = None
        
        # --- Fit the model to the data ---
        # Provide initial guesses for [a, b, T_D, omega]. These may need adjustment depending on data:
        #   a ~ mean of y_data
        #   b ~ (max - min) / 2
        #   T_D ~ half of the span of t_data (or another reasonable guess)
        #   omega ~ initial guess; you may adjust based on expected oscillation frequency.
        a0 = np.mean(y_data)
        b0 = (np.max(y_data) - np.min(y_data)) / 2
        if b0 == 0:
            b0 = 1e-3  # in case data is flat, prevent zero amplitude guess
        # Rough guess for T_D:
        t_span = t_data[-1] - t_data[0]
        T_D0 = t_span / 2 if t_span > 0 else 1.0
        # Rough guess for omega: for example, 2*pi / (total span) if you expect roughly one period over the span:
        omega0 = 2 * np.pi / (t_span if t_span > 0 else 1.0)
        p0 = [a0, b0, T_D0, omega0]
        
        # Perform curve fitting. We pass y_err as sigma to weight the fit.
        try:
            popt, pcov = curve_fit(
                damped_cosine,
                t_data,
                y_data,
                sigma=y_err,
                p0=p0,
                maxfev=10000  # increase max function evaluations if needed
            )
            # Extract fitted parameters
            a_fit, b_fit, T_D_fit, omega_fit = popt
            # Optionally: print or log the fit results
            print(f"Fit for UR={ur}, state={state}: a={a_fit:.4g}, b={b_fit:.4g}, "
                  f"T_D={T_D_fit:.4g}, omega={omega_fit:.4g}")
            
            # Generate a smooth curve for plotting the fit
            t_fit = np.linspace(t_data.min(), t_data.max(), 500)
            y_fit = damped_cosine(t_fit, *popt)
            
            # Plot the fitted curve with same color, semi-transparent
            ax.plot(
                t_fit,
                y_fit,
                linestyle='-',
                color=color,
                alpha=0.5,
                label=None  # do not add extra legend entry for each fit; if desired, adjust
            )
        except Exception as e:
            # If fitting fails, print a warning and continue
            print(f"Warning: could not fit UR={ur}, state={state}. Error: {e}")
            # Optionally, one could plot a flat line or skip.
    
    # Axis labels
    ax.set_xlabel(r'$\#\ \text{of CZ}$', fontdict=font_properties)
    ax.set_ylabel('Fidelity',             fontdict=font_properties)
    ax.tick_params(axis='both', labelsize=18)
    
    # Legend: 3 columns, centered above the plot
    legend = ax.legend(
        ncol=5,
        fontsize=14,
        loc='upper center',
        bbox_to_anchor=(0.5, 1.15),      # just above the top of the axes
        frameon=False
    )
    
    # Figure title (on top of legend)
    fig.suptitle(
        f'X: {ur}',
        fontdict=font_properties,
        y=.85               # push it above the legend
    )
    
    # Make room for title + legend and tighten up the rest
    fig.tight_layout(rect=[0, 0, 1, 0.3])
    
    # Save and close
    filename = f"{experiment_date}_{ur.replace('/', '_')}-fit.pdf"
    fig.savefig(filename, dpi=300)
    plt.close(fig)
    
    print(f"Saved {filename}")


Fit for UR=None, state=00: a=0.998, b=-0.000355, T_D=7668, omega=0.07227
Fit for UR=None, state=01: a=0.6083, b=0.3892, T_D=431.4, omega=1.412e-05
Fit for UR=None, state=0+: a=0.07233, b=0.9238, T_D=250.4, omega=-1.005e-05
Fit for UR=None, state=+0: a=0.4846, b=0.502, T_D=381.4, omega=0.06564
Fit for UR=None, state=++: a=0.3364, b=0.658, T_D=68.45, omega=0.04796


/var/folders/yr/16q32_5s6wq1hdg67qj9yr140000gn/T/ipykernel_7591/3004227366.py:143: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  fig.tight_layout(rect=[0, 0, 1, 0.3])


Saved 06-01-25-CZ_None-fit.pdf
Fit for UR=Both, state=00: a=0.7976, b=0.1995, T_D=39.75, omega=6.23e-06
Fit for UR=Both, state=01: a=0.6761, b=0.3207, T_D=148.1, omega=-5.012e-06
Fit for UR=Both, state=0+: a=0.7203, b=0.2769, T_D=49.38, omega=3.011e-07
Fit for UR=Both, state=+0: a=0.6812, b=0.3125, T_D=99.32, omega=9.552e-07
Fit for UR=Both, state=++: a=0.6796, b=0.3136, T_D=56.74, omega=-3.796e-08
Saved 06-01-25-CZ_Both-fit.pdf
Fit for UR=0, state=00: a=-2.3, b=3.296, T_D=957.3, omega=1.088e-05
Fit for UR=0, state=01: a=0.8293, b=0.181, T_D=50, omega=0.06283
Fit for UR=0, state=0+: a=-0.1094, b=1.106, T_D=159.7, omega=-4.129e-06
Fit for UR=0, state=+0: a=0.8771, b=0.1171, T_D=89.04, omega=3.331e-06
Fit for UR=0, state=++: a=0.05797, b=0.9369, T_D=132.8, omega=1.309e-07


/var/folders/yr/16q32_5s6wq1hdg67qj9yr140000gn/T/ipykernel_7591/3004227366.py:89: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/scipy/optimize/_minpack_py.py:975: RuntimeWarning: divide by zero encountered in divide
  transform = 1.0 / sigma


Saved 06-01-25-CZ_0-fit.pdf
Fit for UR=1, state=00: a=0.812, b=0.1853, T_D=47.91, omega=5.276e-06
Fit for UR=1, state=01: a=0.8067, b=0.1914, T_D=48.6, omega=-6.171e-07
Fit for UR=1, state=0+: a=0.8448, b=0.1502, T_D=50.97, omega=5.912e-06
Fit for UR=1, state=+0: a=0.3546, b=0.6394, T_D=119.8, omega=0.01267
Fit for UR=1, state=++: a=0.2553, b=0.7381, T_D=103.3, omega=0.01031
Saved 06-01-25-CZ_1-fit.pdf
